In [2]:
import torch
import eigenpro.kernels as kernels
import eigenpro.models.sharded_kernel_machine as skm
import eigenpro.run as run
import eigenpro.utils.device as dev

In [3]:
n, p, d, c = 500, 100, 5, 2

Z = torch.randn(p, d)
X_train, X_test = torch.randn(n//2, d), torch.randn(n//2, d)
W_star = torch.randn(d, c)
Y_train, Y_test = X_train @ W_star, X_test @ W_star

dtype = torch.float32
kernel_fn = lambda x, z: kernels.laplacian(x, z, bandwidth=20.)
device = dev.Device.create(use_gpu_if_available=True)
model = skm.create_sharded_kernel_machine(
    Z, c, kernel_fn, device, dtype=dtype, tmp_centers_coeff=2)

sd, sm, qd, qm = 10, 10, 3, 3 # configuration for EigenPro preconditioners
model = run.run_eigenpro(model, X_train, Y_train, X_test, Y_test, device,
                         dtype=dtype, kernel=kernel_fn, s_data=sd, s_model=sm,
                         q_data=qd, q_model=qm, epochs=2,
                         accumulated_gradients=True)

notice: the current implementation can only support 1 GPU, we only use the following device: (cuda:0) 
╒══════════════════════════════════╤═════════╕
│ Configuration                    │   Value │
╞══════════════════════════════════╪═════════╡
│ size of model                    │ 100     │
├──────────────────────────────────┼─────────┤
│ ambient dimension                │ 5       │
├──────────────────────────────────┼─────────┤
│ output dimension                 │ 2       │
├──────────────────────────────────┼─────────┤
│ size of data preconditioner      │ 10      │
├──────────────────────────────────┼─────────┤
│ level of data preconditioner     │ 3       │
├──────────────────────────────────┼─────────┤
│ size of model preconditioner     │ 10      │
├──────────────────────────────────┼─────────┤
│ level of model preconditioner    │ 3       │
├──────────────────────────────────┼─────────┤
│ size of training dataset         │ 250     │
├──────────────────────────────────┼─────────┤
│ cr

Projection:   0%|                                                                                                                   | 0/3 [00:00<?, ?it/s]
                                                                                                                                                          
Projection:   0%|                                                                                                                   | 0/3 [00:00<?, ?it/s]
                                                                                                                                                          
Projection:   0%|                                                                                                                   | 0/3 [00:00<?, ?it/s]
                                                                                                                                                          
Epoch 1/2: 100%|██████████████████████████████████████████████████████

╒═══════════════════╤══════════════╕
│ Epoch 1 Summary   │ Value        │
╞═══════════════════╪══════════════╡
│ Test Loss         │ 0.3331757486 │
├───────────────────┼──────────────┤
│ Test Accuracy     │ 95.20%       │
╘═══════════════════╧══════════════╛


Projection:   0%|                                                                                                                   | 0/3 [00:00<?, ?it/s]
                                                                                                                                                          
Projection:   0%|                                                                                                                   | 0/3 [00:00<?, ?it/s]
                                                                                                                                                          
Projection:   0%|                                                                                                                   | 0/3 [00:00<?, ?it/s]
                                                                                                                                                          
Epoch 2/2: 100%|██████████████████████████████████████████████████████

╒═══════════════════╤══════════════╕
│ Epoch 2 Summary   │ Value        │
╞═══════════════════╪══════════════╡
│ Test Loss         │ 0.2300972044 │
├───────────────────┼──────────────┤
│ Test Accuracy     │ 98.80%       │
╘═══════════════════╧══════════════╛
